# K-Nearest Neighbors

K-NN is an effective model for performing classification, which may help us make sense of the clusters that we saw from
`exploring_data.ipynb`. In particular
![PCA with 2 Principal Components](/images/pca_2_components.png)
and
![PCA with 3 Principal Components](/images/pca_3_components.png).

There is some concern with this. Chiefly, we did not see separateable clusters from the 2d PCA representation
of the data, or from our vantage of the 3d representation. It is my hope, however, that there is more cluster separation
offered in a higher dimension. We will see.

### Goals
1. Determine if cluster separation is viable in a higher dimension.
    * This goes back to the point from above. The clusters appeared to be on top of each other in 2D and (possibly) 3D.
    Perhaps this is not the case in a higher dimensional setting.
2. Determine how effective K-NN will be on the data.
    * This means that we are not tuning, but exploring a few sets of options to get a feel.
3. Determine what distance metric appears to make the most sense.
    * Euclidean distance does not make much intuitive sense when comparing wines. Perhaps there's a better metric.

#### Implementing KNN
As done in `SVM.ipynb`, I will perform KNN on the data after standardization and PCA has been applied, as it will
generally improves performance to run KNN on a lower dimensional dataset. This may not be required, however, as we're
looking at low-dimensional data as is ($d \leq 11$), and a kernel would make this an $O(d)$ operation. We will use
8 principal components for the time being, as that is what the exploratory analysis indicated to be effective.

#### Loading the Data

In [31]:
from models.data_loader import DataLoader
from sklearn.neighbors import KNeighborsClassifier
import itertools
import numpy as np
import pandas as pd

rs = np.random.RandomState(42069)

# Load the data into memory and apply PCA with 8 principal components
dl = DataLoader('../data/winequality-red.csv', random_state=rs)
dl.apply_pca_to_dataset(8)

# Apply a train/test split.
X_train, X_test, y_train, y_test = dl.train_test_split()

# Define the parameters of the model to explore.
n_neighbors = [5, 25, 50, 100, 200, 500]  # The goal is to explore a wide set of neighbors to get some intuition.
distance_metrics = [
    'euclidean',   # L2 norm
    'manhattan',   # L1 norm
    'minkowski',   # Lp norm.
    'chebyshev',   # max(|x - y|)
]

N_train,d = X_train.shape

#### Applying the exploratory Models to the Data.
By taking a set product of $\mathrm{n_neighbors} \times \mathrm{distance_metrics}$, we can feel out how well the model
is performing, while also getting an idea of which distance metric(s) might make sense to explore further.

In [37]:
performances = pd.DataFrame(columns=['n_neighbors', 'distance_metric', 'accuracy'])

for k, distance_metric in itertools.product(n_neighbors, distance_metrics):
    knn = KNeighborsClassifier(n_neighbors=k, metric=distance_metric, p=d)
    knn.fit(X_train, y_train)

    # Compute the model's accuracy on the testing data.
    accuracy = knn.score(X_test, y_test)

    performances = performances.append({
        'n_neighbors': k,
        'distance_metric': distance_metric,
        'accuracy': accuracy
    }, ignore_index=True)

#### Display the Top-10 Best Models

In [38]:
performances = performances.sort_values('accuracy', ascending=False)
print(performances.head(10))

   n_neighbors distance_metric  accuracy
5           25       manhattan  0.591667
4           25       euclidean  0.589583
10          50       minkowski  0.587500
11          50       chebyshev  0.585417
0            5       euclidean  0.577083
7           25       chebyshev  0.577083
12         100       euclidean  0.575000
6           25       minkowski  0.575000
14         100       minkowski  0.572917
15         100       chebyshev  0.572917


#### What was learned?
* Using 25-50 nearest neighbors appeared to be most effective.
    * this provides a good indication of where to begin with hyper parameter tuning, but it does not give us much
    conclusive information.
* All four distance metrics showed up in the top 5 model predictions, and all performed similarly.
    * This is interesting, as all of these top performing models appeared within ~2% of each other. This implies a few things.
        1. A different performance metric will be required to evaluate the models. Perhaps AUC would be a good idea.
        2. The notion of distance used does not appear to have much impact on the model. In other words, the ways in
            which we attempt to numerically classify which wines are the "most similar" does not appear to have a great
            effect.
            * Perhaps this has something to do with the data being standardized ahead of time; however, with distances
            being relative, I do not believe this is an issue.
        3. It may make sense to pursue other types of distance metrics. For example, Mahalanobis Distance, which has
            some effective interpretations.

#### Exploring Mahalanobis Distance
Recommended reading: [Mahalanobis Distance](https://en.wikipedia.org/wiki/Mahalanobis_distance).

In [44]:
import warnings
warnings.filterwarnings('ignore')
# Mahalanobis Distance requires the covariance matrix
Sigma = np.cov(X_train)

print("Exploring using Mahalanobis Distance: ")
print("n_neighbors\taccuracy")
for k in n_neighbors:
    knn = KNeighborsClassifier(algorithm='brute', n_neighbors=k, metric='mahalanobis', metric_params={'V': Sigma})
    knn.fit(X_train, y_train)

    accuracy = knn.score(X_test, y_test)
    print(f"{k}\t\t{accuracy}")

Exploring using Mahalanobis Distance: 
n_neighbors	accuracy
5		0.5770833333333333
25		0.5458333333333333
50		0.5708333333333333
100		0.5625
200		0.55625
500		0.5145833333333333


Not much of a performance increase, unfortunately. In fact, it generally performed worse than the previous models, with
the best accuracy only being $~57.71\%$.

